<a href="https://colab.research.google.com/github/dsapandora/DEMUCS/blob/master/DemucsFacebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uso de la libreria DEMUCS para separar pistas de un archivo de audio.

Lo primero que se debe hacer es la instalacion de **Anaconda**, que funcionara como sistema de gestion de liberias para el proyecto. (Esta primera etapa no es necesaria si ya tiene instalado ANACONDA en sus maquinas)
## Descargar el software ananconda.

In [2]:
!wget https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh && bash Anaconda3-5.2.0-Linux-x86_64.sh -bfp /usr/local


--2020-06-05 06:24:43--  https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 651745206 (622M) [application/x-sh]
Saving to: ‘Anaconda3-5.2.0-Linux-x86_64.sh’

Anaconda3-5.2.0-Lin 100%[===================>] 621.55M   158MB/s    in 4.0s    

2020-06-05 06:24:47 (156 MB/s) - ‘Anaconda3-5.2.0-Linux-x86_64.sh’ saved [651745206/651745206]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: blas-1.0-mkl ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: intel-openmp-2018.0.0-8 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libgfortran-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: bzip2-1.0.6

## Configurar el software Anaconda para ser cargado en e el entorno de Google Colab

In [0]:
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

## Verificar si la instalacion ocurrio correctamente

In [5]:
!conda info --all
!conda list


     active environment : None
       user config file : /root/.condarc
 populated config files : 
          conda version : 4.5.4
    conda-build version : 3.10.5
         python version : 3.6.5.final.0
       base environment : /usr/local  (writable)
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/free/linux-64
                          https://repo.anaconda.com/pkgs/free/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://repo.anaconda.com/pkgs/pro/linux-64
                          https://repo.anaconda.com/pkgs/pro/noarch
          package cache : /usr/local/pkgs
                          /root/.conda/pkgs
       envs directories : /usr/local/envs
                          /root/.conda/envs
               platfor

# DEMUCS
Demucs se basa en la arquitectura convolucional U-Net inspirada en Wave-U-Net y SING, con GLU, un BiLSTM entre el codificador y el decodificador, la inicialización específica de pesos y las convoluciones transpuestas en el decodificador.

Conv-Tasnet es un modelo de separación desarrollado para el habla que predice una máscara en una representación lineal sobrecompleta aprendida utilizando un modelo puramente convolucional con zancada de 1 y bloques convolucionales dilatados. Reutilizamos el código del repositorio kaituoxu / Conv-TasNet y agregamos soporte para múltiples canales de audio.

![Como Funciona](https://github.com/facebookresearch/demucs/raw/master/demucs.png)

Para usuarlo debemos habilitarlos por usando anaconda


In [10]:
!git clone https://github.com/facebookresearch/demucs


Cloning into 'demucs'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 2832 (delta 9), reused 3 (delta 1), pack-reused 2812
Receiving objects: 100% (2832/2832), 48.61 MiB | 28.31 MiB/s, done.
Resolving deltas: 100% (196/196), done.


In [15]:
! echo ". /usr/local/etc/profile.d/conda.sh" >> ~/.bashrc
! echo "conda activate" >> ~/.bashrc
! cd ./demucs
! conda install -y -q  git
! conda env update -f environment-gpu.yml
! conda activate demucs

Solving environment: ...working... done

# All requested packages already installed.


SpecNotFound: Can't process without a name


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
If your shell is Bash or a Bourne variant, enable conda for the current user with

    $ echo ". /usr/local/etc/profile.d/conda.sh" >> ~/.bashrc

or, for all users, enable conda with

    $ sudo ln -s /usr/local/etc/profile.d/conda.sh /etc/profile.d/conda.sh

The options above will permanently enable the 'conda' command, but they do NOT
put conda's base (root) environment on PATH.  To do so, run

    $ conda activate

in your terminal, or to put the base environment on PATH permanently, run

    $ echo "conda activate" >> ~/.bashrc

Previous to conda 4.4, the recommended way to activate conda was to modify PATH in
your ~/.bashrc file.  You should manually remove the line that looks like

    export PATH="/usr/local/bin:$PATH"

^^^ The above line should NO LONGER be i

Una vez activado podemos proceder con las pruebas-
Necesitaremos un archivo mp3 para poder hacer la prueba asi que procedemos de descargarlo.
Luego ejecutamos el programa

---



In [16]:
!wget https://files.freemusicarchive.org/storage-freemusicarchive-org/music/WFMU/junior85/Left_For_Deadish/junior85_-_Left_For_Deadish.mp3

!python -m demucs.separate --dl -n demucs --mp3 junior85_-_Left_For_Deadish.mp3
import argparse
import gzip
import json
import sys
from collections import defaultdict
from pathlib import Path

import numpy as np
import treetable as tt

BASELINES = [
    'WaveUNet',
    'MMDenseLSTM',
    'OpenUnmix',
    'IRM2',
]
EVALS = Path("evals")
LOGS = Path("logs")
BASELINE_EVALS = Path("baselines")
STD_KEY = "seed"

parser = argparse.ArgumentParser("result_table.py")
parser.add_argument("-p",
                    "--paper",
                    action="store_true",
                    help="show results from the paper experiment")
parser.add_argument("-i", "--individual", action="store_true", help="no aggregation by seed")
parser.add_argument("-l", "--latex", action="store_true", help="output easy to copy latex")
parser.add_argument("metric", default="SDR", nargs="?")
args = parser.parse_args()

if args.paper:
    EVALS = Path("results/evals")
    LOGS = Path("results/logs")


def read_track(metric, results, pool=np.nanmedian):
    all_metrics = {}
    for target in results["targets"]:
        source = target["name"]
        metrics = [frame["metrics"][metric] for frame in target["frames"]]
        metrics = pool(metrics)
        all_metrics[source] = metrics
    return all_metrics


def read(metric, path, pool=np.nanmedian):
    all_metrics = defaultdict(list)
    for f in path.iterdir():
        if f.name.endswith(".json.gz"):
            results = json.load(gzip.open(f, "r"))
            metrics = read_track(metric, results, pool=pool)
            for source, value in metrics.items():
                all_metrics[source].append(value)
    return {key: np.array(value) for key, value in all_metrics.items()}


all_stats = defaultdict(list)
for name in BASELINES:
    all_stats[name] = [read(args.metric, BASELINE_EVALS / name / "test")]

for path in EVALS.iterdir():
    results = path / "results" / "test"
    if not results.exists():
        continue
    if not args.paper and not (LOGS / (path.name + ".done")).exists():
        continue
    name = path.name
    model = "Demucs"
    if "tasnet" in name:
        model = "Tasnet"
    if name == "default":
        parts = []
    else:
        parts = [p.split("=") for p in name.split(" ") if "tasnet" not in p]
    if not args.individual:
        parts = [(k, v) for k, v in parts if k != STD_KEY]
    name = model + " " + " ".join(f"{k}={v}" for k, v in parts)
    stats = read(args.metric, results)
    if (not stats or len(stats["drums"]) != 50):
        print(f"Missing stats for {results}", file=sys.stderr)
    else:
        all_stats[name].append(stats)

metrics = [tt.leaf("score", ".2f"), tt.leaf("std", ".2f")]
sources = ["drums", "bass", "other", "vocals"]

mytable = tt.table([tt.leaf("name"), tt.group("all", metrics + [tt.leaf("count")])] +
                   [tt.group(source, metrics) for idx, source in enumerate(sources)])

lines = []
for name, stats in all_stats.items():
    line = {"name": name}
    if 'accompaniment' in stats:
        del stats['accompaniment']
    alls = []
    for source in sources:
        stat = [np.nanmedian(s[source]) for s in stats]
        alls.append(stat)
        line[source] = {"score": np.mean(stat), "std": np.std(stat) / len(stat)**0.5}
    alls = np.array(alls)
    line["all"] = {
        "score": alls.mean(),
        "std": alls.mean(0).std() / alls.shape[1]**0.5,
        "count": alls.shape[1]
    }
    lines.append(line)


def latex_number(m):
    out = f"{m['score']:.2f}"
    if m["std"] > 0:
        std = "{:.2f}".format(m["std"])[1:]
        out += f" $\\scriptstyle\\pm {std}$"
    return out


lines.sort(key=lambda x: -x["all"]["score"])
if args.latex:
    for line in lines:
        cols = [
            line['name'],
            latex_number(line["all"]),
            latex_number(line["drums"]),
            latex_number(line["bass"]),
            latex_number(line["other"]),
            latex_number(line["vocals"])
        ]
        print(" & ".join(cols) + r" \\")
else:
    print(tt.treetable(lines, mytable, colors=['33', '0']))

--2020-06-05 07:03:08--  https://files.freemusicarchive.org/storage-freemusicarchive-org/music/WFMU/junior85/Left_For_Deadish/junior85_-_Left_For_Deadish.mp3
Resolving files.freemusicarchive.org (files.freemusicarchive.org)... 172.67.73.168, 104.26.11.182, 104.26.10.182, ...
Connecting to files.freemusicarchive.org (files.freemusicarchive.org)|172.67.73.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5238149 (5.0M) [audio/mpeg]
Saving to: ‘junior85_-_Left_For_Deadish.mp3’

junior85_-_Left_For 100%[===================>]   5.00M  --.-KB/s    in 0.06s   

2020-06-05 07:03:08 (84.8 MB/s) - ‘junior85_-_Left_For_Deadish.mp3’ saved [5238149/5238149]

/usr/local/bin/python: No module named demucs.separate


ModuleNotFoundError: ignored